# Water

## **Step 0**: Setup

In [1]:
import numpy as np

# Qiskit circuit libraries
from qiskit.circuit.library import EfficientSU2

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# PySCF itself
import pyscf

# Convert to QCSchema
from chemist_converter import pyscf_to_qcschema, qcschema_to_electronic_structure

# Chemistry transformations
from passes.transformations import ActiveSpaceTransformer
from passes.mappings import (ElectronicStructureToFermionicHamiltonian,
                             JordanWignerMapper,
                             ParityMapper,
                             BravyiKitaevMapper)

# Fulqrum workflow
from fulqrum import Workflow

# SPSA
from spsa import minimize_spsa

## **Step 1** Map the problem to a Quantum Native format (Set of Operators, and a set of Quantum Circuits)

### PySCF to QCSchema

In [2]:
mol = pyscf.M(atom = 'O 0.0 0.0 0.115; H 0.0 0.754 -0.459; H 0.0 -0.754 -0.459',
              basis = 'sto-3g',
              charge=0,
              spin=0,
              )

mf = pyscf.scf.RHF(mol).run(verbose=False)
schema = pyscf_to_qcschema(mf)

/Users/paul/mambaforge/envs/qiskit/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


### Electronic structure to qubit hamiltonian

In [3]:
elec_struct  = qcschema_to_electronic_structure(schema)

chem_workflow = Workflow([ActiveSpaceTransformer(2, 3),
                          ElectronicStructureToFermionicHamiltonian(),
                          #JordanWignerMapper(),
                          BravyiKitaevMapper(),
                        ])

hamiltonian = chem_workflow.run(electronic_structure=elec_struct)
hamiltonian
#reduced_elec_struct = ActiveSpaceTransformer(2, 3).run(elec_struct)
#fermi_hamiltonian = ElectronicStructureToFermionicHamiltonian().run(reduced_elec_struct)
#hamiltonian = BravyiKitaevMapper().run(fermi_hamiltonian)

SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIZZ', 'IIIZII', 'IIZZZI', 'IZIIII', 'ZZIIII', 'IIIIZI', 'IIIZIZ', 'IIZZZZ', 'IZIIIZ', 'ZZIIIZ', 'XYYZIX', 'XXXIZX', 'XYYZZX', 'XXXIIX', 'YIYXXX', 'XZXYYX', 'YIYYXY', 'XZXXYY', 'IIIZZZ', 'IIZZIZ', 'IZIIZZ', 'ZZIIZZ', 'ZXIYYZ', 'IXIYYZ', 'ZXIXXI', 'IXIXXI', 'IIZIZI', 'IZIZII', 'ZZIZII', 'IZZZZI', 'ZZZZZI', 'ZIIIII'],
              coeffs=[-0.05135748+0.j, -0.17628591+0.j, -0.42585723+0.j, -0.48094051+0.j,
 -0.17628591+0.j, -0.42585723+0.j, -0.48094051+0.j,  0.13748638+0.j,
  0.15056812+0.j,  0.22003977+0.j,  0.14721913+0.j,  0.15665676+0.j,
 -0.00973275+0.j, -0.00973275+0.j,  0.00973275+0.j,  0.00973275+0.j,
 -0.00608864+0.j,  0.00608864+0.j, -0.00608864+0.j, -0.00608864+0.j,
  0.11275929+0.j,  0.14721913+0.j,  0.14925818+0.j,  0.14173508+0.j,
 -0.02897579+0.j,  0.02897579+0.j, -0.02897579+0.j,  0.02897579+0.j,
  0.15665676+0.j,  0.14173508+0.j,  0.1553773 +0.j,  0.13748638+0.j,
  0.15056812+0.j,  0.11275929+0.j])

### Ansatz circuit

In [4]:
ansatz = EfficientSU2(num_qubits=hamiltonian.num_qubits, reps=1, entanglement='linear')

## **Step 2**: Optimize the circuits and the operators to be measured

None here, letting the primitives handle it.

## **Step 3**: Execute using a quantum primitive function (estimator or sampler)


In [5]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [6]:
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

In [7]:
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=250)

In [8]:
res

 message: Optimization terminated successfully.
 success: True
     fun: -1.6486296983507056
       x: [ 6.289e+00  3.149e+00 ...  2.782e+00  1.712e+00]
     nit: 250
    nfev: 500

## **Step 4**: Post-processing of the results

In [9]:
res.fun

-1.6486296983507056